In [1]:
import numpy as np

This notebook uses [absorbing Markov chains](https://en.wikipedia.org/wiki/Absorbing_Markov_chain) to solve the following problem: If you repeatedly flip a fair coin, what is the expected number of flips until a particular sequences of heads and tails occurs?

The functions defined here can actually solve the more general problem for a sequence of outcomes where there are more than 2 possible outcomes and the outcomes do not have equal probability.

The first function in this notebook is `expected_number_of_steps`, which takes as input a transition matrix and starting state (represented as an integer indexing a row in the matrix), and returns the expected number of steps until an absorbing state is reached. This function uses `np.linalg.solve` to solve a system of linear equations derived from the input data.

In [2]:
def expected_number_of_steps(transition_matrix, starting_state):
    #First, find the set of absorbing states.
    n=transition_matrix.shape[0]
    absorbing_states=[]
    for i in range(n):
        row=np.zeros(n)
        row[i]=1
        if (row==transition_matrix[i]).all():
            absorbing_states.append(i)
    if len(absorbing_states)==0:
        raise Exception("The input transition matrix defines a Markov chain with no absorbing states.")
    if starting_state in absorbing_states:
        return 0
    
    #Delete the rows and columns corresponding to the absorbing states.
    Q=np.delete(transition_matrix, absorbing_states, axis=0)
    Q=np.delete(Q, absorbing_states, axis=1)
    
    ss=starting_state
    #Correctly index the starting state if its index is after an absorbing state's index.
    for i in absorbing_states:
        if ss > i:
            ss = ss-1
    
    return np.linalg.solve(np.identity(n-1)-Q,np.repeat(1,n-1))[ss]

Now, given a target sequence of n outcomes, and probabilities for those outcomes, we can consider the following Markov chain: 
* The states are the integers from 0 to n. You are in state k if k is the largest integer such that your sequence of outcomes ends in the first k outcomes in the target sequence. In other words, it represents how many steps you are into the target sequence.
* If you are in state n, you always stay in state n. In other words, the process stops once the target sequence occurs.
* If j < n, the probability of going from state j to state k is the probability that if you add an extra outcome to the first j outcomes of the target sequence, this new sequence will sequence will be in state k. There is always a probability of going from j to j+1. But there is also a chance of going backwards.

Given a sequence of outcomes, and a dictionary of probabilities for those outcomes, the function `get_transition_matrix_from_seq` will return the transition matrix associated to the Markov chain described above.

In [3]:
def matching_subseq_length(main_seq,other_seq):
    #Returns the largest k such that the last k symbols of the other sequence
    #are the first k symbols of the main sequence.
    #other_seq is assumed to be no longer than main_seq.
    m=len(other_seq)
    for k in range(m,-1,-1):
        if main_seq[:k]==other_seq[-k:]:
            return k
    return 0

def get_transition_matrix_from_seq(seq, prob_dict):
    n=len(seq)
    P=np.zeros((n+1,n+1))
    P[n][n]=1
    for i in range(n):
        s=seq[:i]
        for c in prob_dict.keys():
            P[i][matching_subseq_length(seq,s+c)]+=prob_dict[c]
    return P

Here is an example. Suppose you are flipping a coin, and the target sequence is 3 heads in a row. Then the states are 0, 1, 2, 3. In states 0-2, there is a 1/2 probability of advancing one state, and a 1/2 probability of going back to 0. In state 3, you always stay at 3.

The function `get_transition_matrix_from_seq` will return the transition matrix corresponding to this system, which has the probabilities described above:

In [4]:
fair_coin={"H":0.5,"T":0.5}
get_transition_matrix_from_seq("HHH",fair_coin)

array([[0.5, 0.5, 0. , 0. ],
       [0.5, 0. , 0.5, 0. ],
       [0.5, 0. , 0. , 0.5],
       [0. , 0. , 0. , 1. ]])

Finally, `ev_to_reach_seq` will combine both functions above to directly solve the original problem. Given a sequence of outcomes, and a dictionary of probabilities for those outcomes, it returns the expected number of steps for the target sequence to occur.

In [5]:
def ev_to_reach_seq(seq, prob_dict):
    P=get_transition_matrix_from_seq(seq, prob_dict)
    return expected_number_of_steps(P,0)

Now, let's use this function to find the expected number of coin flips to reach some different sequences of heads and tails:

In [6]:
coin_flip_seqs=["H","HH", "HT", "HHH","HHT","HTH","HTT"]
for s in coin_flip_seqs:
    print(s+': '+str(ev_to_reach_seq(s,fair_coin)))

H: 2.0
HH: 6.0
HT: 4.0
HHH: 14.0
HHT: 8.0
HTH: 10.0
HTT: 8.0


Let's see what happens if the coin is unfair and has a 2/3 probability of landing on heads.

In [7]:
unfair_coin={'H':2/3,'T':1/3}
for s in coin_flip_seqs:
    print(s+': '+str(ev_to_reach_seq(s,unfair_coin)))

H: 1.4999999999999998
HH: 3.749999999999999
HT: 4.499999999999999
HHH: 7.124999999999998
HHT: 6.749999999999998
HTH: 8.249999999999996
HTT: 13.499999999999996


To conclude, here are some examples with 3 outcomes of equal probability:

In [8]:
three_equal={c:1/3 for c in ["A","B","C"]}
for s in ["A","AA","AB","AAA","AAB","ABA","BAA","ABC"]:
    print(str(s)+": "+str(ev_to_reach_seq(s,three_equal)))

A: 2.9999999999999996
AA: 11.999999999999998
AB: 8.999999999999995
AAA: 38.999999999999986
AAB: 26.999999999999986
ABA: 29.999999999999947
BAA: 26.999999999999943
ABC: 26.999999999999943
